使用之前5天的数据，对之后48小时的空气质量进行预测，模型如下

![](http://p3rz3gu1u.bkt.clouddn.com/2018-04-19-seq2seq_model.png)
<caption><center> **Figure 1**: lstm model</center></caption>

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np 
import seaborn as sns
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

from utils.plot_util import plot_forecast_and_actual_example
from metrics.metrics import SMAPE_on_dataset_v1
from seq2seq.seq2seq_data_util import get_training_statistics, generate_training_set, generate_dev_set
from seq2seq.multi_variable_seq2seq_model_parameters import build_graph

%load_ext autoreload
%autoreload 2

/home/cvdev/tf/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

gpu_config = tf.ConfigProto()
gpu_config.gpu_options.allow_growth = True
session = tf.Session(config=gpu_config)
KTF.set_session(session)

### 2. Args

多变量版本

In [3]:
station_list = ['dongsi_aq','tiantan_aq','guanyuan_aq','wanshouxigong_aq','aotizhongxin_aq',
            'nongzhanguan_aq','wanliu_aq','beibuxinqu_aq','zhiwuyuan_aq','fengtaihuayuan_aq',
            'yungang_aq','gucheng_aq','fangshan_aq','daxing_aq','yizhuang_aq','tongzhou_aq',
            'shunyi_aq','pingchang_aq','mentougou_aq','pinggu_aq','huairou_aq','miyun_aq',
            'yanqin_aq','dingling_aq','badaling_aq','miyunshuiku_aq','donggaocun_aq',
            'yongledian_aq','yufa_aq','liulihe_aq','qianmen_aq','yongdingmennei_aq',
            'xizhimenbei_aq','nansanhuan_aq','dongsihuan_aq']            
X_aq_list = ["PM2.5","PM10","O3","CO","SO2","NO2"]  
y_aq_list = ["PM2.5","PM10","O3"]
X_meo_list = ["temperature","pressure","humidity","direction","speed/kph"]
use_day=True
pre_days=5
batch_size=128

少变量版本（测试）

In [4]:
# station_list = ['aotizhongxin_aq']            
# X_aq_list = ["PM2.5","PM10","O3","CO","SO2","NO2"]  
# y_aq_list = ["PM2.5"]
# X_meo_list = ["temperature","pressure","humidity","direction","speed/kph"]
# use_day = True
# pre_days = 5
# batch_size = 128

### 3. Prepare test datasets in 3-D format - (batch_size, time_step, feature_dim)

In [5]:
test_x, test_y = generate_dev_set(station_list=station_list,
                                  X_aq_list=X_aq_list, 
                                  y_aq_list=y_aq_list, 
                                  X_meo_list=None,
                                  pre_days=pre_days)

In [6]:
print(test_x.shape, test_y.shape)

(17, 120, 210) (17, 48, 105)


尝试生成一个训练样本，确保是我们想要的尺寸。

In [7]:
X_training_batch, y_training_batch = generate_training_set(station_list=station_list,
                                                           X_aq_list=X_aq_list,
                                                           y_aq_list=y_aq_list,
                                                           pre_days=pre_days,
                                                           X_meo_list=None,
                                                           use_day=True,
                                                           batch_size=batch_size)
print(X_training_batch.shape, y_training_batch.shape)

(128, 120, 210) (128, 48, 105)


### 4. Build the model and train the model 

In [8]:
input_seq_len = pre_days * 24
output_seq_len = 48
hidden_dim = 512
input_dim = 210
output_dim = 105
num_stacked_layers = 3

learning_rate=1e-3
lambda_l2_reg=0.003
GRADIENT_CLIPPING=2.5
total_iteractions = 2000
KEEP_RATE = 0.5

In [9]:
rnn_model = build_graph(feed_previous=False, 
                        input_seq_len=input_seq_len, 
                        output_seq_len=output_seq_len, 
                        hidden_dim=hidden_dim, 
                        input_dim=input_dim, 
                        output_dim=output_dim, 
                        num_stacked_layers=num_stacked_layers, 
                        learning_rate=learning_rate,
                        lambda_l2_reg=lambda_l2_reg,
                        GRADIENT_CLIPPING=GRADIENT_CLIPPING)

In [ ]:
train_losses = []
val_losses = []

saver = tf.train.Saver()

init = tf.global_variables_initializer()
with tf.Session() as sess:

    sess.run(init)
    losses = []
    print("Training losses: ")
    for i in range(total_iteractions):
        batch_input, batch_output = generate_training_set(station_list,
                                                          X_aq_list,
                                                          y_aq_list,
                                                          X_meo_list=None,
                                                          use_day=use_day,
                                                          pre_days=pre_days,
                                                          batch_size=batch_size)

        
        feed_dict = {rnn_model['enc_inp'][t]: batch_input[:,t,:] for t in range(input_seq_len)}
        feed_dict.update({rnn_model['target_seq'][t]: batch_output[:,t,:] for t in range(output_seq_len)})
        _, loss_t = sess.run([rnn_model['train_op'], rnn_model['loss']], feed_dict) 
        
        if i%10 == 0:
            print("loss after %d/%d iteractions : %.3f" %(i, total_iteractions, loss_t))
            
            # 想要对训练过程中训练集的 smape 进行监督，发现模型并不是处在“预测”的状态，因此放弃
            # train_preds = sess.run(rnn_model['reshaped_outputs'], feed_dict)
            # train_preds = [np.expand_dims(pred, 1) for pred in train_preds]
            # train_preds = np.concatenate(train_preds, axis = 1)
            
        losses.append(loss_t)
        
    temp_saver = rnn_model['saver']()
    save_path = temp_saver.save(sess, os.path.join('./seq2seq/new_multi_variable_model_results/', 'multivariate_ts_pollution_case'))
        
print("Checkpoint saved at: ", save_path)

Training losses: 
loss after 0/2000 iteractions : 151.999
loss after 10/2000 iteractions : 79.616
loss after 20/2000 iteractions : 74.756
loss after 30/2000 iteractions : 68.966
loss after 40/2000 iteractions : 65.922
loss after 50/2000 iteractions : 64.076
loss after 60/2000 iteractions : 62.529
loss after 70/2000 iteractions : 60.709
loss after 80/2000 iteractions : 59.410
loss after 90/2000 iteractions : 57.951
loss after 100/2000 iteractions : 56.729
loss after 110/2000 iteractions : 55.182
loss after 120/2000 iteractions : 53.705
loss after 130/2000 iteractions : 52.419
loss after 140/2000 iteractions : 50.912
loss after 150/2000 iteractions : 49.722
loss after 160/2000 iteractions : 47.996
loss after 170/2000 iteractions : 47.406
loss after 180/2000 iteractions : 46.253
loss after 190/2000 iteractions : 45.130
loss after 200/2000 iteractions : 44.166
loss after 210/2000 iteractions : 42.739
loss after 220/2000 iteractions : 41.732
loss after 230/2000 iteractions : 40.680
loss aft

In [ ]:
%matplotlib inline
plt.plot(losses)

## Inference on test 
Notice the batch prediction which is different to previous

In [ ]:
rnn_model = build_graph(feed_previous=True, 
                        input_seq_len=input_seq_len, 
                        output_seq_len=output_seq_len, 
                        hidden_dim=hidden_dim, 
                        input_dim=input_dim, 
                        output_dim=output_dim, 
                        num_stacked_layers=num_stacked_layers, 
                        learning_rate=learning_rate,
                        lambda_l2_reg=lambda_l2_reg,
                        GRADIENT_CLIPPING=GRADIENT_CLIPPING)

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:

    sess.run(init)
    
    saver = rnn_model['saver']().restore(sess,  os.path.join('./seq2seq/new_multi_variable_model_results/', 'multivariate_ts_pollution_case'))
    
    feed_dict = {rnn_model['enc_inp'][t]: test_x[:, t, :] for t in range(input_seq_len)} # batch prediction
    feed_dict.update({rnn_model['target_seq'][t]: np.zeros([test_x.shape[0], output_dim], dtype=np.float32) for t in range(output_seq_len)})
    final_preds = sess.run(rnn_model['reshaped_outputs'], feed_dict)
    
    final_preds = [np.expand_dims(pred, 1) for pred in final_preds]
    final_preds = np.concatenate(final_preds, axis = 1)

In [ ]:
print("Shape of predictions is ",final_preds.shape)

### Example of many featutres

In [ ]:
output_features = []
for station in station_list : 
    for aq_feature in y_aq_list :
        output_features.append(station + "_" + aq_feature)

# 特征要和训练时候的特征顺序保持一致
output_features.sort()

In [ ]:
print(output_features)
print("Number of features is : ", len(output_features))

In [ ]:
# 预测值普遍在 O3 上表现较好，另外两个参数　PM2.5 和　PM10 上通常捕捉不到高频分量
for i in range(len(output_features)):
    plot_forecast_and_actual_example(test_x, test_y, final_preds, output_features, index=0, feature_index=i)

### 某个特征在整个dev数据集时间跨度上的表现

In [ ]:
feature_index = 0
test_y_expand = np.concatenate([test_y[i,:,feature_index] for i in range(0, test_y.shape[0])], axis = 0)
final_preds_expand = np.concatenate([final_preds[i,:,feature_index] for i in range(0, final_preds.shape[0])], axis = 0)
plt.plot(final_preds_expand, color = 'orange', label = 'predicted')
plt.plot(test_y_expand, color = 'blue', label = 'actual')
plt.title("test data - one month")
plt.legend(loc="upper left")
plt.show()

### Smapes of all features

载入训练样本的统计量

In [ ]:
statistics = get_training_statistics()

In [ ]:
statistics

计算 smape

In [ ]:
aver_smapes, smapes_of_features = SMAPE_on_dataset_v1(test_y, final_preds, output_features, statistics, 1)

In [ ]:
# smape value on all features
smapes_of_features

In [ ]:
print("The average smape on all features in the dev set is : ",aver_smapes)

- epoch - aver_smape relationship
    - 50 0.85
    - 1000 0.75

# ChangeLog
- 0427 v0
    - 完成了第一版本模型